<a href="https://colab.research.google.com/github/JLRRC/MASTERIA/blob/main/03MIAR%20Algoritmos%20de%20Optimizacio%CC%81n/seminario_algoritmos_Sesiones_de_doblaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario - Sesiones de doblaje<br>
**Nombre y Apellidos:** Jesús Lozano Rodríguez  <br>
**Url:** JLRRC/MASTERIA/03MIAR_Algoritmos_de_Optimización/Seminario_Algoritmos.ipynb <br>
Problema:
>1. Sesiones de doblaje <br>

**Descripción del problema:**

**Problema 1. Organizar sesiones de doblaje**

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran toda la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10 <br>
Número de tomas : 30 <br>
Actores/Tomas : https://bit.ly/36D8IuK <br>
- 1 indica que el actor participa en la toma <br>
- 0 en caso contrario <br>

(*) La respuesta es obligatoria
                                        

**Respuesta**

In [7]:
#Heurística voraz con validación de factibilidad.
#Es un problema de asignación combinatoria donde un algoritmo voraz bien diseñado puede encontrar
#soluciones razonablemente óptimas en poco tiempo sin requerir métodos complejos como programación
#entera o metaheurísticas costosas.

import numpy as np
import pandas as pd

# Definir la matriz de participación de actores en las tomas
matriz_tomas = [
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 0, 0, 1, 0, 0],
]

# Convertimos la matriz en un array de numpy para facilitar cálculos
matriz_tomas_np = np.array(matriz_tomas)

# Conjunto de tomas que aún no han sido asignadas a un día
tomas_pendientes = set(range(len(matriz_tomas)))

# Lista donde se almacenarán los días y sus tomas asignadas
dias = []

# Algoritmo voraz para la asignación de tomas en días de grabación
while tomas_pendientes:
    dia_actual = set()

    # Ordenamos tomas pendientes por la cantidad de actores involucrados
    for toma in sorted(tomas_pendientes, key=lambda t: np.sum(matriz_tomas_np[t]), reverse=True):
        # Condición: No más de 6 tomas por día y evitar conflictos de actores
        if len(dia_actual) < 6 and (len(dia_actual) == 0 or np.all(matriz_tomas_np[list(dia_actual)].sum(axis=0) + matriz_tomas_np[toma] <= 1)):
            dia_actual.add(toma)

    # Guardamos la asignación del día
    dias.append(sorted(dia_actual))

    # Eliminamos las tomas ya asignadas
    tomas_pendientes -= dia_actual

# Crear un DataFrame con la planificación final
dias_dict = {f"Día {i+1}": sesiones for i, sesiones in enumerate(dias)}
df_doblaje = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in dias_dict.items()]))

# Mostrar la planificación
print("\nPlanificación optimizada de sesiones de doblaje:")
print(df_doblaje)


Planificación optimizada de sesiones de doblaje:
   Día 1  Día 2  Día 3  Día 4  Día 5  Día 6  Día 7  Día 8  Día 9
0    0.0      1    2.0    4.0    5.0    6.0    9.0    7.0    8.0
1    NaN      3    NaN    NaN    NaN    NaN    NaN    NaN    NaN


**Modelo para el espacio de soluciones**<br>
**(*) 1.¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)**


**Respuesta**

La mejor estructura de datos para este problema es una matriz binaria, donde las filas representan las tomas y las columnas representan los actores.
* Cada celda de la matriz contiene un 1 si el actor participa en la toma y 0 en caso contrario.
* Esto facilita la verificación de restricciones y la agrupación de tomas en sesiones óptimas.
* Alternativamente, un grafo bipartito también podría ser útil, con actores y tomas como nodos y conexiones si un actor participa en una toma.

**Según el modelo para el espacio de soluciones**<br>
**(*)2. ¿Cual es la función objetivo?**

**(*)3. ¿Es un problema de maximización o minimización?**

**Respuesta**

La función objetivo es minimizar el número de días de grabación, asegurando que cada actor solo participe cuando sea necesario para reducir costos.

Es un problema de minimización, ya que buscamos reducir el número total de días de grabación.

**4. Diseña un algoritmo para resolver el problema por fuerza bruta**

**Respuesta**

In [2]:
# El enfoque de fuerza bruta probaría todas las posibles maneras de asignar las tomas a los días. Esto implicaría:
#	1.	Generar todas las combinaciones posibles de grupos de tomas que cumplan con la restricción de no más de 6 tomas por día.
#	2.	Verificar en cada combinación que los actores coincidan correctamente.
#	3.	Elegir la asignación con el menor número de días.

from itertools import permutations

def fuerza_bruta(tomas, max_tomas_dia=6):
    min_dias = float('inf')
    mejor_asignacion = None

    for perm in permutations(tomas):
        dias = []
        temp = []

        for toma in perm:
            if len(temp) < max_tomas_dia:
                temp.append(toma)
            else:
                dias.append(temp)
                temp = [toma]

        if temp:
            dias.append(temp)

        if len(dias) < min_dias:
            min_dias = len(dias)
            mejor_asignacion = dias

    return mejor_asignacion

print(fuerza_bruta(list(range(30))))


**5. Calcula la complejidad del algoritmo por fuerza bruta**

**Respuesta**

El algoritmo de fuerza bruta genera todas las permutaciones posibles de tomas y las agrupa en días, por lo que su complejidad es O(n!) (factorial). Es completamente impracticable para más de 10-12 tomas.

**(*)6. Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**

**Respuesta**

In [8]:
#Podemos usar un algoritmo heurístico voraz, donde:
#	1.	Se ordenan las tomas según el número de actores involucrados.
#	2.	Se asignan iterativamente en días respetando la restricción de 6 tomas por día.
#	3.	Se verifica que los actores no se repitan en diferentes días innecesariamente.
import numpy as np

def heuristica_voraz(matriz_tomas):
    tomas_pendientes = set(range(len(matriz_tomas)))
    dias = []

    while tomas_pendientes:
        dia_actual = set()

        for toma in sorted(tomas_pendientes, key=lambda t: np.sum(matriz_tomas[t]), reverse=True):
            if len(dia_actual) < 6 and np.all(matriz_tomas[list(dia_actual)].sum(axis=0) + matriz_tomas[toma] <= 1):
                dia_actual.add(toma)

        dias.append(sorted(dia_actual))
        tomas_pendientes -= dia_actual

    return dias

# Ejecutar el algoritmo
matriz_tomas = np.random.randint(0, 2, (30, 10))  # Simulación de la matriz
print(heuristica_voraz(matriz_tomas))

[[23], [5, 9], [11], [4], [13], [22], [0], [1], [6], [12], [28], [8], [10, 27], [14], [17, 19], [18], [20, 21], [24], [29], [2], [3], [15, 16], [26], [7], [25]]


**(*)7. Calcula la complejidad del algoritmo**

**Respuesta**

La heurística voraz tiene una complejidad de O(n² log n) debido a la ordenación inicial de las tomas en O(n log n), seguida de la asignación iterativa de tomas en O(n²) en el peor caso. Además, la verificación de restricciones de compatibilidad tiene un costo de O(m), lo que puede influir dependiendo del problema.

Esto representa una mejora significativa respecto a la complejidad de O(n!) de la fuerza bruta, que evaluaría todas las posibles asignaciones de tomas a días.

**8. Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**

**Respuesta**

In [5]:
#Podemos generar una matriz binaria aleatoria de 30 tomas y 10 actores:
import numpy as np

def generar_datos():
    return np.random.randint(0, 2, (30, 10))

datos = generar_datos()
print(datos)

[[0 0 1 1 1 0 1 0 0 0]
 [1 0 1 1 0 1 0 0 1 0]
 [0 0 0 1 0 1 0 0 1 0]
 [0 0 0 1 1 1 0 1 0 0]
 [0 0 1 1 0 0 1 0 1 0]
 [0 1 1 0 0 0 1 0 1 0]
 [1 0 0 1 0 1 1 1 1 1]
 [0 1 0 1 1 1 1 1 1 0]
 [0 1 1 1 1 0 1 1 1 1]
 [1 0 0 1 0 1 0 0 1 0]
 [1 1 0 0 0 0 0 1 1 1]
 [0 1 0 0 1 1 1 1 0 0]
 [1 0 0 0 0 1 1 1 0 1]
 [0 0 1 1 0 1 1 0 1 0]
 [0 1 1 1 1 1 0 1 1 1]
 [1 0 1 1 1 0 0 0 0 0]
 [1 1 1 0 1 0 0 1 1 1]
 [0 0 1 1 0 1 1 1 0 1]
 [1 1 0 1 1 0 0 1 1 0]
 [1 0 0 0 0 1 1 0 1 0]
 [0 0 0 1 0 0 0 0 0 1]
 [0 0 1 0 1 1 0 1 0 1]
 [0 1 0 0 0 1 0 1 0 1]
 [0 1 0 0 0 1 1 0 0 1]
 [1 0 1 0 1 0 0 0 0 1]
 [0 0 1 1 1 1 1 0 1 1]
 [1 1 1 0 0 1 1 0 1 1]
 [0 0 1 1 0 1 1 1 0 1]
 [0 1 1 0 1 1 0 1 1 0]
 [1 0 0 1 0 1 1 0 0 1]]


**9. Aplica el algoritmo al juego de datos generado**

**Respuesta**

In [6]:
resultado = heuristica_voraz(datos)
print(resultado)

[[8], [14], [6], [7], [16], [25], [26], [17], [18], [27], [20, 28], [1], [0, 10], [11], [12], [13], [21], [29], [3, 5], [4, 22], [9], [15, 23], [19], [2, 24]]


**10. Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**

**Respuesta**

* Documentación de NumPy y Pandas.
* Notas del curso sobre heurísticas.
* Implementaciones de algoritmos voraces en optimización combinatoria.

**11. Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**

**Respuesta**

* Se podría usar Programación Entera para obtener una solución óptima garantizada.
* Se pueden probar metaheurísticas como búsqueda tabú o recocido simulado para mejorar la solución.
* Si el problema escala, podría aplicarse aprendizaje automático para predecir la mejor agrupación de tomas.